In [1]:
import pandas as pd

In [2]:
all_takeoff = pd.read_csv('../initial_takeoff_data/merge_initial_takeoff_data.ipynb')

In [3]:
# 불필요한 피쳐 제거

all_takeoff = all_takeoff.drop(["출/도착구분", "출발공항코드", "출발공항명", "도착공항코드", "도착공항명", "예상시간"], axis=1)

In [4]:
# 여객 항공편만을 취급

all_takeoff = all_takeoff[all_takeoff['구분'] == '여객']

In [6]:
# 결측치 제거

all_takeoff = all_takeoff.dropna()

In [10]:
# 출발시간 컬럼에 올바른 형식이 아닌 데이터를 찾기 위한 정규 표현식

invalid_time = all_takeoff[~all_takeoff['출발시간'].str.match(r'^\d{2}:\d{2}$')]

# 결과 출력
print(invalid_time[['날짜', '항공사', '편명', '계획시간', '출발시간', "구분", "현황"]])

              날짜       항공사      편명   계획시간 출발시간  구분  현황
985     20240102      대한항공   KE719  20:40    :  여객  취소
998     20240102    아시아나항공   OZ178  21:20    :  여객  취소
1336    20240103    중국남방항공   CZ688  13:20    :  여객  취소
1602    20240103    중국남방항공   CZ672  21:55    :  여객  취소
2113    20240104      대한항공  KE8051  20:30    :  여객  취소
...          ...       ...     ...    ...  ...  ..  ..
203573  20241216     핀란드항공    AY42  23:00    :  여객  취소
204145  20241217     핀란드항공    AY42  23:00    :  여객  취소
204182  20241218  에어아시아버하드  AK1624  01:05    :  여객  지연
206324  20241221     폴란드항공   LO098  13:25    :  여객  취소
210682  20241228     캐나다항공   AC064  16:55    :  여객  취소

[497 rows x 7 columns]


In [12]:
## timestamp 특징 생성 ##

# 잘못된 출발시간을 찾는 정규 표현식
invalid_time = all_takeoff[~all_takeoff['출발시간'].str.match(r'^\d{2}:\d{2}$')]

# 잘못된 출발시간을 확인 (예: :만 있는 값들)
print(invalid_time[['날짜', '항공사', '편명', '계획시간', '출발시간', '구분', '현황']])

# `timestamp` 생성 함수
def create_timestamp(row):
    # 출발시간이 올바른 형태(HH:MM)인지 확인
    if pd.to_datetime(row['출발시간'], errors='coerce') is pd.NaT:
        # 출발시간이 잘못된 경우 예상시간 사용
        return pd.to_datetime(f"{row['날짜']} {row['계획시간']}", format="%Y%m%d %H:%M")
    else:
        # 출발시간이 올바른 경우 출발시간 사용
        return pd.to_datetime(f"{row['날짜']} {row['출발시간']}", format="%Y%m%d %H:%M")

# 잘못된 출발시간을 가진 행은 예측된 시간으로 처리하고, 나머지는 그대로 처리
all_takeoff['timestamp'] = all_takeoff.apply(create_timestamp, axis=1)

# 결과 확인
print(all_takeoff[['날짜', '계획시간', '출발시간', 'timestamp']])

              날짜       항공사      편명   계획시간 출발시간  구분  현황
985     20240102      대한항공   KE719  20:40    :  여객  취소
998     20240102    아시아나항공   OZ178  21:20    :  여객  취소
1336    20240103    중국남방항공   CZ688  13:20    :  여객  취소
1602    20240103    중국남방항공   CZ672  21:55    :  여객  취소
2113    20240104      대한항공  KE8051  20:30    :  여객  취소
...          ...       ...     ...    ...  ...  ..  ..
203573  20241216     핀란드항공    AY42  23:00    :  여객  취소
204145  20241217     핀란드항공    AY42  23:00    :  여객  취소
204182  20241218  에어아시아버하드  AK1624  01:05    :  여객  지연
206324  20241221     폴란드항공   LO098  13:25    :  여객  취소
210682  20241228     캐나다항공   AC064  16:55    :  여객  취소

[497 rows x 7 columns]
              날짜   계획시간   출발시간           timestamp
0       20240101  00:05  00:01 2024-01-01 00:01:00
2       20240101  00:05  00:05 2024-01-01 00:05:00
3       20240101  00:15  00:51 2024-01-01 00:51:00
4       20240101  00:15  00:18 2024-01-01 00:18:00
5       20240101  00:20  01:54 2024-01-01 01:54:00
...       

In [13]:
## 분정보 초기화

# timestamp에서 분을 00으로 변경하는 방법
all_takeoff['timestamp'] = all_takeoff['timestamp'].dt.floor('H')

# 결과 확인
print(all_takeoff[['날짜', '출발시간', 'timestamp']])

              날짜   출발시간           timestamp
0       20240101  00:01 2024-01-01 00:00:00
2       20240101  00:05 2024-01-01 00:00:00
3       20240101  00:51 2024-01-01 00:00:00
4       20240101  00:18 2024-01-01 00:00:00
5       20240101  01:54 2024-01-01 01:00:00
...          ...    ...                 ...
212660  20241231  23:33 2024-12-31 23:00:00
212662  20241231  23:48 2024-12-31 23:00:00
212663  20241231  23:26 2024-12-31 23:00:00
212667  20241231  23:51 2024-12-31 23:00:00
212669  20241231  23:54 2024-12-31 23:00:00

[179719 rows x 3 columns]


/var/folders/m1/bqb2x_hs2p3c9xfm8sbxkgcr0000gn/T/ipykernel_5794/1901965016.py:4: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  all_takeoff['timestamp'] = all_takeoff['timestamp'].dt.floor('H')


In [19]:
all_takeoff.to_csv("./2024_takeoff.csv", index=False)